In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

# Cargar PNGs con canal alfa
chaplin_hat = cv2.imread("images/chaplin_hat.png", cv2.IMREAD_UNCHANGED)
chaplin_mustache = cv2.imread("images/chaplin_moustache.png", cv2.IMREAD_UNCHANGED)
chaplin_letfeye = cv2.imread("images/chaplin_left_eye.png", cv2.IMREAD_UNCHANGED)
chaplin_righteye = cv2.imread("images/chaplin_right_eye.png", cv2.IMREAD_UNCHANGED)


mtcnn_det = MTCNN()
video_capture = cv2.VideoCapture(0)

def overlay(fondo, img, x, y):
    fh, fw = fondo.shape[:2]
    ih, iw = img.shape[:2]

    if x >= fw or y >= fh:
        return fondo

    # calcular límites 
    w = min(iw, fw - x)
    h = min(ih, fh - y)

    # recortar imagen si se sale
    img = img[0:h, 0:w]

    # separar canales
    overlay_rgb = img[:, :, :3]
    alpha = img[:, :, 3:] / 255.0

    #montar imagen
    fondo[y:y+h, x:x+w] = (1 - alpha) * fondo[y:y+h, x:x+w] + alpha * overlay_rgb
    return fondo


while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = mtcnn_det.detect_faces(rgb_frame)

    if len(results) == 0:
        # si no hay cara mostrar sin filtro
        cv2.imshow("Chaplin filter", cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        if cv2.waitKey(1) & 0xFF == 27:
            break
        continue

    #cojer primera cara detectada
    keypoints = results[0]["keypoints"]

    left_eye = keypoints["left_eye"]
    right_eye = keypoints["right_eye"]
    mouth_left = keypoints["mouth_left"]
    mouth_right = keypoints["mouth_right"]

    # Centro de ojos
    eye_center_x = int((left_eye[0] + right_eye[0]) / 2)
    eye_center_y = int((left_eye[1] + right_eye[1]) / 2)

    eye_distance = abs(right_eye[0] - left_eye[0])

    #sombrero
    hat_width = int(3 * eye_distance) 
    hat_height = int(hat_width * chaplin_hat.shape[0] / chaplin_hat.shape[1])

    x_hat = eye_center_x - hat_width // 2
    y_hat = eye_center_y - int(hat_height * 1.25)

    hat_resize = cv2.resize(chaplin_hat, (hat_width, hat_height))
    frame = overlay(frame, hat_resize, x_hat, y_hat)

    #bigote
    mouth_width = abs(mouth_right[0] - mouth_left[0])
    mustache_width = int(0.9 * mouth_width) 
    mustache_height = int(mustache_width * chaplin_mustache.shape[0] / chaplin_mustache.shape[1])

    mouth_center_x = int((mouth_left[0] + mouth_right[0]) / 2)
    mouth_center_y = int((mouth_left[1] + mouth_right[1]) / 2)

    x_m = mouth_center_x - mustache_width // 2
    y_m = mouth_center_y - mustache_height  

    mustache_resize = cv2.resize(chaplin_mustache, (mustache_width, mustache_height))
    frame = overlay(frame, mustache_resize, x_m, y_m)

    # cejas
    eyebrow_width = int(1.6 * abs(right_eye[0] - left_eye[0]) * 0.5)
    eyebrow_height = int(eyebrow_width * chaplin_letfeye.shape[0] / chaplin_letfeye.shape[1])

    # Resize
    left_eyebrow_r = cv2.resize(chaplin_letfeye, (eyebrow_width, eyebrow_height))
    right_eyebrow_r = cv2.resize(chaplin_righteye, (eyebrow_width, eyebrow_height))

    x_left_eyebrow = int(left_eye[0] - eyebrow_width/1.8)
    y_left_eyebrow = int(left_eye[1] - eyebrow_height*1)

    x_right_eyebrow = int(right_eye[0] - eyebrow_width/2)
    y_right_eyebrow = int(right_eye[1] - eyebrow_height*1.1)

    #  overlay
    frame = overlay(frame, left_eyebrow_r,  x_left_eyebrow,  y_left_eyebrow)
    frame = overlay(frame, right_eyebrow_r, x_right_eyebrow, y_right_eyebrow)

    
    # Mostrar en gris
    background_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    background_gray = cv2.cvtColor(background_gray, cv2.COLOR_GRAY2BGR)

    cv2.imshow('Chaplin filter', background_gray)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video_capture.release()
cv2.destroyAllWindows()


ValueError: operands could not be broadcast together with shapes (173,264,1) (0,264,3) 

: 